In [1]:
import numpy as np
import pandas as pd
import datetime
from scipy import integrate

In [2]:
df = pd.read_csv("data/20231229_sensor_data.csv")

In [3]:
df.head()

,t,online,relay_status,countdown_1,switch_1,cur_voltage,cur_current,cycle_time,cur_power,add_ele,random_time
0,1703808003261,True,power_on,0,True,2264,0,NaN,0,1,NaN
1,1703836844490,True,power_on,0,True,2264,0,NaN,0,1,NaN
2,1703839844490,True,power_on,0,True,2234,169,NaN,183,4,NaN
3,1703839860366,True,power_on,0,True,2234,169,NaN,183,4,NaN
4,1703839876001,True,power_on,0,True,2234,169,NaN,183,11,NaN


Convert timestamp column 't' to datetime

In [4]:
df['t'] = pd.to_datetime(df['t']/1000., unit='s')
df['elapsed_time'] = (df['t'] - df['t'][0]).dt.total_seconds()
df = df.set_index(['t'])

In [5]:
df.head()

,online,relay_status,countdown_1,switch_1,cur_voltage,cur_current,cycle_time,cur_power,add_ele,random_time,elapsed_time
t,,,,,,,,,,,
2023-12-29 00:00:03.260999936,True,power_on,0,True,2264,0,NaN,0,1,NaN,0.000
2023-12-29 08:00:44.490000128,True,power_on,0,True,2264,0,NaN,0,1,NaN,28841.229
2023-12-29 08:50:44.490000128,True,power_on,0,True,2234,169,NaN,183,4,NaN,31841.229
2023-12-29 08:51:00.365999872,True,power_on,0,True,2234,169,NaN,183,4,NaN,31857.105
2023-12-29 08:51:16.000999936,True,power_on,0,True,2234,169,NaN,183,11,NaN,31872.740


# Create online df

In [6]:
df_online = df.loc[df['online']]

In [7]:
df_online.head()

,online,relay_status,countdown_1,switch_1,cur_voltage,cur_current,cycle_time,cur_power,add_ele,random_time,elapsed_time
t,,,,,,,,,,,
2023-12-29 00:00:03.260999936,True,power_on,0,True,2264,0,NaN,0,1,NaN,0.000
2023-12-29 08:00:44.490000128,True,power_on,0,True,2264,0,NaN,0,1,NaN,28841.229
2023-12-29 08:50:44.490000128,True,power_on,0,True,2234,169,NaN,183,4,NaN,31841.229
2023-12-29 08:51:00.365999872,True,power_on,0,True,2234,169,NaN,183,4,NaN,31857.105
2023-12-29 08:51:16.000999936,True,power_on,0,True,2234,169,NaN,183,11,NaN,31872.740


# Basic visualisations

In [8]:
import plotly.express as px


fig = px.scatter(
    df_online,
    # x='t',
    y='cur_power'
)
fig.show()

fig = px.scatter(
    df_online,
    # x='t',
    y='add_ele'
)
fig.show()

# Calculate energy productions sums

# A) Using numerical integration
The energy produced can be aproximated by integrating the power values in 'cur_power'. 

In [9]:
# energy in Wh
df_online['total_energy_integrated'] = integrate.cumtrapz(df_online['cur_power'].to_numpy(), df_online['elapsed_time'].to_numpy(), initial=0) / 10 / 60 / 60

/tmp/ipykernel_40945/2728737807.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [10]:
df_online['total_energy_integrated']

t
2023-12-29 00:00:03.260999936      0.000000
2023-12-29 08:00:44.490000128      0.000000
2023-12-29 08:50:44.490000128      7.625000
2023-12-29 08:51:00.365999872      7.705703
2023-12-29 08:51:16.000999936      7.785181
                                    ...    
2023-12-29 14:45:16.122999808    125.542274
2023-12-29 14:45:31.653000192    125.542274
2023-12-29 14:45:47.282000128    125.542274
2023-12-29 14:46:02.783000064    125.542274
2023-12-29 14:46:18.280999936    125.542274
Name: total_energy_integrated, Length: 1189, dtype: float64

## B) Using the add_ele signal
After restarting a plug, 'add_ele' contains 1. After 30 minutes the engergy produced in Wh in this time interval is written to this field. This value does not change until another 30 minute interval passes or the sensor is restarted. If the sensor restarts 1 is written for 30 minutes again. If it stayed online the energy produced in the new time interval is written to the field.

Therefore, changes in the 'add_ele' field have to be filtered for and summed up after correcting for the 1 always written, even when no energy was produced. 

This approach should be more exact than numerical integration since the integration here is done directly on the iot plug.

A problem that might appear: The energy produced in two subsequent intervals is the exactly the same. This has to be checked by finding subsequent intervals with no 'add_ele' change, but data points produced in both intervals (aka the sensor was available in both intervals, but 'add_ele' did not change).

In [15]:
# get first row of every changed add_ele
# this signals new energy production sum
# df_ele = df_online.drop_duplicates(subset = ['add_ele'], keep = 'first')
df_ele = df_online.loc[df_online['add_ele'].shift() != df_online['add_ele']]

# check if any ele interval was skipped
# because two subsequent ele values were the same
ele_skipped_mask = (df_ele.index.diff() > datetime.timedelta(minutes=45))
ele_rows_skipped = []

print(df_ele['add_ele'])
print()

if ele_skipped_mask.any():
    prev_index = df_online.index[0]
    for i, (t, row) in enumerate(df_ele.iterrows()):
        if ele_skipped_mask[i]:
            df_filtered = df_online.loc[prev_t:t, :]
            time_gap = t - prev_t
            time_gap_minutes = time_gap.seconds / 60
            print('ele gap detected:')
            print(f"start: {str(prev_t)} - add_ele: {df_ele.loc[prev_t, 'add_ele']}")
            print(f"end: {str(t)} - add_ele: {df_ele.loc[t, 'add_ele']}")
            print(f"{time_gap_minutes} minutes")
            print(df_filtered.shape)

            # calculate delta between 
            energy_integrated = df_ele.loc[t, 'total_energy_integrated'] - df_ele.loc[prev_t, 'total_energy_integrated']
            energy_ele = df_ele.loc[prev_t, 'add_ele'] - 1

            print(f"energy_integrated: {energy_integrated}")
            print(f"energy_ele: {energy_ele}")

            # if energy_ele == 0:
            #     print("Previous energy was 0")
            #     print("Skipping...")
            #     print()
            #     continue
            if df_filtered.shape[0] <= 2:
                print("No data points between changes.")
                print("Skipping...")
                print()

            # calculate ratio
            if energy_ele > 0:
                n_energy = round(energy_integrated / energy_ele) - 1
                n_time = round(time_gap_minutes / 30) - 1
                print(f"n_energy: {n_energy}")
                print(f"n_time: {n_time}")
            else:
                print("Energy == 0")
                n_time = 1
                print(f"n_time: {n_time}")


            add_energy_ele = n_time * energy_ele
            add_energy_integrated = energy_integrated
            print(f"add_energy_ele: {add_energy_ele}")
            print(f"add_energy_integrated: {add_energy_integrated}")

            print("Adding new rows...")
            for j in range(n_time):
                new_row_index = int((j+1) / (n_time+1) * (df_filtered.shape[0]-1))
                print(f"{j+1} / {n_time} : index {new_row_index} / {df_filtered.shape[0]}")

                # new_row = pd.DataFrame(df_filtered.iloc[new_row_index], columns=df_filtered.columns, index=[])
                new_row = pd.DataFrame(df_filtered.iloc[new_row_index]).transpose()
                print(new_row)
                ele_rows_skipped.append(new_row)


        prev_t = t

df_ele = pd.concat([df_ele] + ele_rows_skipped, axis=0).sort_index(ascending=True)
print(df_ele['add_ele'])

# cumulate energy values and store in 'ele_cumsum'
df_ele['ele_cumsum'] = (df_ele['add_ele'] - 1).cumsum()

t
2023-12-29 00:00:03.260999936     1
2023-12-29 08:50:44.490000128     4
2023-12-29 08:51:16.000999936    11
2023-12-29 09:21:29.676999936    10
2023-12-29 09:51:25.792999936    23
2023-12-29 10:21:48.680000000     6
2023-12-29 10:51:26.096000000    10
2023-12-29 11:51:36.488000000    14
2023-12-29 12:51:34.002000128     8
2023-12-29 13:21:42.288000000     4
2023-12-29 13:51:31.229000192     9
2023-12-29 14:21:37.884000000     6
Name: add_ele, dtype: int64

ele gap detected:
start: 2023-12-29 00:00:03.260999936 - add_ele: 1
end: 2023-12-29 08:50:44.490000128 - add_ele: 4
530.6833333333333 minutes
(3, 13)
energy_integrated: 7.625
energy_ele: 0
Energy == 0
n_time: 1
add_energy_ele: 0
add_energy_integrated: 7.625
Adding new rows...
1 / 1 : index 1 / 3
                              online relay_status countdown_1 switch_1  \
2023-12-29 08:00:44.490000128   True     power_on           0     True   

                              cur_voltage cur_current cycle_time cur_power  \
2023-12-29 08

In [16]:
fig = px.scatter(
    df_ele,
    y='add_ele'
)
fig.show()

fig = px.scatter(
    df_ele,
    y='ele_cumsum'
)
fig.show()

In [21]:
df_online['total_energy_ele'] = df_ele['ele_cumsum']
df_online['total_energy_ele'] = df_online['total_energy_ele'].astype(float).interpolate(method='linear')

/tmp/ipykernel_40945/1460182139.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_40945/1460182139.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
df_online['total_energy_ele'] = df_ele['ele_cumsum']
df_online['total_energy_ele'] = df_online['total_energy_ele'].fillna(method='ffill')

In [22]:
fig = px.scatter(
    df_online,
    y='total_energy_integrated'
)
fig.show()

fig = px.scatter(
    df_online,
    y='total_energy_ele'
)
fig.show()

In [27]:
grouping = df_online.groupby([df_online.index.hour])
hourly = grouping[['total_energy_integrated', 'total_energy_ele']].max() - grouping[['total_energy_integrated', 'total_energy_ele']].min()

In [31]:
import plotly.express as px

fig = px.bar(hourly, y=['total_energy_integrated', 'total_energy_ele'], barmode='group')
fig.show()